# 🎨 MangaGen - AI Manga Generation Pipeline

Generate complete manga pages with consistent characters from text prompts!

**Features:**
- 📝 Story → Scene JSON (Gemini 2.0 Flash)
- 🎨 SDXL + IP-Adapter for consistent characters
- 💬 Smart dialogue bubble placement
- 📄 PDF output with zip download

---

## 📋 Prerequisites

Before running, add these secrets in **Kaggle Settings → Add-ons → Secrets**:
1. `GEMINI_API_KEY` - Get from https://aistudio.google.com/app/apikey
2. `HF_TOKEN` (optional) - HuggingFace token for model downloads

---

## 🔧 Cell 1: Clone Repository & Install Dependencies

In [ ]:
%%bash
# Clone the repository
cd /kaggle/working
if [ -d "manga-gen" ]; then
    echo "Repository already exists, pulling latest..."
    cd manga-gen && git pull
else
    echo "Cloning repository..."
    git clone https://github.com/Barun-2005/manga-gen-ai-pipeline.git manga-gen
    cd manga-gen
fi
echo ""
echo "Latest commit:"
git log -1 --oneline

In [ ]:
# Install dependencies using our Kaggle-specific script
!cd /kaggle/working/manga-gen && bash install_kaggle_deps.sh

## 🔑 Cell 2: Set Up API Keys

In [ ]:
import os
import sys

# Add manga-gen to path
sys.path.insert(0, '/kaggle/working/manga-gen')
os.chdir('/kaggle/working/manga-gen')

# Load API keys from Kaggle Secrets
try:
    from kaggle_secrets import UserSecretsClient
    secrets = UserSecretsClient()
    
    # Required: Gemini API Key
    os.environ['GEMINI_API_KEY'] = secrets.get_secret('GEMINI_API_KEY')
    print("✅ GEMINI_API_KEY loaded from Kaggle secrets")
    
    # Optional: HuggingFace Token
    try:
        os.environ['HF_TOKEN'] = secrets.get_secret('HF_TOKEN')
        print("✅ HF_TOKEN loaded")
    except:
        print("ℹ️ HF_TOKEN not set (optional)")
        
except Exception as e:
    print(f"⚠️ Could not load secrets: {e}")
    print("")
    print("🔧 To add secrets:")
    print("   1. Click 'Add-ons' menu at top")
    print("   2. Select 'Secrets'")
    print("   3. Add 'GEMINI_API_KEY' with your API key")
    print("")
    print("Get your Gemini API key at: https://aistudio.google.com/app/apikey")

## ⚙️ Cell 3: Configuration

Customize your manga generation settings here!

In [ ]:
# ============================================
# 🎨 MANGA CONFIGURATION - EDIT THIS!
# ============================================

# Your story prompt - describe your manga scene
STORY_PROMPT = """
Astra, a determined space scavenger with messy silver hair and a grease-stained orange jumpsuit, 
explores a derelict spaceship. She finds a glowing blue artifact in the cockpit.
""".strip()

# Visual style: "bw_manga" (black & white) or "color_anime" (colorful)
STYLE = "bw_manga"

# Panel layout: "2x2" (4 panels), "vertical_webtoon" (3 panels), "3_panel", "single"
LAYOUT = "2x2"

# Generation quality (higher = better but slower)
INFERENCE_STEPS = 30  # 20-50 recommended
GUIDANCE_SCALE = 7.5  # 6-9 recommended

# ============================================

print("📋 Configuration:")
print(f"   Story: {STORY_PROMPT[:80]}...")
print(f"   Style: {STYLE}")
print(f"   Layout: {LAYOUT}")
print(f"   Steps: {INFERENCE_STEPS}")
print(f"   Guidance: {GUIDANCE_SCALE}")

## 📝 Cell 4: Generate Scene Plan (Gemini)

In [ ]:
import json

# Generate scene plan using Gemini
!python scripts/generate_scene_json.py "{STORY_PROMPT}" --style {STYLE} --layout {LAYOUT} --output scene_plan.json

# Display the generated scene plan
print("\n" + "="*50)
print("📋 Generated Scene Plan")
print("="*50)

if os.path.exists('scene_plan.json'):
    with open('scene_plan.json', 'r') as f:
        scene_plan = json.load(f)
    
    print(f"\nTitle: {scene_plan.get('title', 'Untitled')}")
    print(f"Style: {scene_plan.get('style', 'unknown')}")
    print(f"Layout: {scene_plan.get('layout', 'unknown')}")
    
    print(f"\n📚 Characters ({len(scene_plan.get('characters', []))})")
    for char in scene_plan.get('characters', []):
        print(f"   • {char['name']}: {char['hair_color']} hair, {char['clothing'][:50]}...")
    
    print(f"\n🖼️ Panels ({len(scene_plan.get('panels', []))})")
    for panel in scene_plan.get('panels', []):
        print(f"   {panel['panel_number']}. [{panel['camera_angle']}] {panel['description'][:60]}...")
        for d in panel.get('dialogue', []):
            print(f"      💬 {d['speaker']}: \"{d['text'][:40]}...\"")
else:
    print("❌ Scene plan generation failed. Check the error above.")

## 🖼️ Cell 5: Generate Panel Images (GPU)

This is the main image generation step. Uses:
- SDXL for high-quality anime/manga style
- IP-Adapter for character consistency

**Time estimate:** ~2-5 minutes for 4 panels on Kaggle T4 GPU

In [ ]:
import torch

# Check GPU availability
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"✅ GPU Available: {gpu_name} ({gpu_mem:.1f} GB)")
else:
    print("⚠️ No GPU detected - using mock mode")

# Create output directory
os.makedirs('outputs', exist_ok=True)

# Generate panels
# Remove --mock for real generation, add it back for testing
!python scripts/generate_panels.py \
    --scene scene_plan.json \
    --output outputs/ \
    --steps {INFERENCE_STEPS} \
    --guidance {GUIDANCE_SCALE}

In [ ]:
# Display generated panels
from IPython.display import display, Image as IPImage
import glob

print("\n🖼️ Generated Panels:")
print("="*50)

panels = sorted(glob.glob('outputs/panel_*.png'))
for panel in panels:
    print(f"\n{os.path.basename(panel)}")
    display(IPImage(filename=panel, width=400))

# Show character references if they exist
refs = glob.glob('outputs/character_refs/*.png')
if refs:
    print("\n📸 Character References:")
    for ref in refs:
        print(f"\n{os.path.basename(ref)}")
        display(IPImage(filename=ref, width=200))

## 💬 Cell 6: Place Dialogue Bubbles

In [ ]:
# Calculate bubble positions
!python scripts/place_bubbles.py \
    --panels outputs/ \
    --scene scene_plan.json \
    --output bubbles.json

# Display bubble data
if os.path.exists('bubbles.json'):
    with open('bubbles.json', 'r') as f:
        bubbles = json.load(f)
    
    print("\n💬 Bubble Placements:")
    for panel_key, panel_bubbles in bubbles.items():
        print(f"   {panel_key}: {len(panel_bubbles)} bubble(s)")

## 📄 Cell 7: Compose Final Page & PDF

In [ ]:
# Compose final page with bubbles
!python scripts/compose_page.py \
    --panels outputs/ \
    --bubbles bubbles.json \
    --scene scene_plan.json \
    --output outputs/

# Display final page
if os.path.exists('outputs/manga_page.png'):
    print("\n🎨 Final Manga Page:")
    display(IPImage(filename='outputs/manga_page.png', width=600))
    
    # Show file info
    print("\n📁 Output Files:")
    for f in ['outputs/manga_page.png', 'outputs/manga_page.pdf', 'manga_output.zip']:
        if os.path.exists(f):
            size_mb = os.path.getsize(f) / 1024 / 1024
            print(f"   ✅ {f} ({size_mb:.2f} MB)")

## 📦 Cell 8: Download Your Manga!

Click the download link below to get your manga!

In [ ]:
from IPython.display import FileLink, display
import shutil

# Copy zip to working directory for easy download
if os.path.exists('manga_output.zip'):
    shutil.copy('manga_output.zip', '/kaggle/working/manga_output.zip')
    print("\n" + "="*50)
    print("🎉 YOUR MANGA IS READY!")
    print("="*50)
    print("\n📥 Click below to download:")
    display(FileLink('/kaggle/working/manga_output.zip'))
    print("\n📋 Zip contains:")
    print("   • manga_page.pdf - Final manga with bubbles")
    print("   • manga_page.png - Full resolution page")
    print("   • panel_*.png - Individual panels")
    print("   • panel_*_with_bubbles.png - Panels with dialogue")
    print("   • character_refs/ - Character reference images")
else:
    print("⚠️ Output zip not found. Check previous cells for errors.")
    print("\nYou can manually download individual files from the file browser.")

---

## 🔄 Bonus: Generate Another Page

Want to generate more? Just edit the `STORY_PROMPT` in Cell 3 and run cells 4-8 again!

In [ ]:
# Quick regeneration with new prompt
NEW_PROMPT = """
Kira, a young samurai with a scar over her left eye and traditional red hakama, 
faces a demon in the rain-soaked streets of ancient Kyoto.
""".strip()

# Uncomment to regenerate:
# !python scripts/generate_scene_json.py "{NEW_PROMPT}" --style bw_manga --layout 3_panel --output scene_plan.json
# !python scripts/generate_panels.py --scene scene_plan.json --output outputs/ --steps 30
# !python scripts/place_bubbles.py --panels outputs/ --scene scene_plan.json --output bubbles.json
# !python scripts/compose_page.py --panels outputs/ --bubbles bubbles.json --scene scene_plan.json --output outputs/

---

## 📊 Pipeline Summary

This notebook ran the complete MangaGen pipeline:

| Step | Component | Time | Status |
|------|-----------|------|--------|
| 1 | Clone & Install | ~1 min | ✅ |
| 2 | API Keys | instant | ✅ |
| 3 | Scene Plan (Gemini) | ~5 sec | ✅ |
| 4 | Panel Generation (SDXL+IP-Adapter) | ~2-5 min | ✅ |
| 5 | Bubble Placement | ~10 sec | ✅ |
| 6 | PDF Composition | ~5 sec | ✅ |
| 7 | Download Zip | instant | ✅ |

**Total time:** ~3-7 minutes

---

Made with ❤️ by Barun | [GitHub](https://github.com/Barun-2005/manga-gen-ai-pipeline)